In [3]:
import pandas as pd
import numpy as np
import folium 
from shapely.geometry import Point
import geopandas as gpd
from cartiflette import carti_download

In [4]:
# =================================================================
# 1. CHARGEMENT DES DÉPARTEMENTS ET PRÉPARATION
# =================================================================

# Chargement des départements
departements = carti_download(
    values=["France"],
    crs=4326,
    borders="DEPARTEMENT",
    vectorfile_format="geojson",
    simplification=50,
    filter_by="FRANCE_ENTIERE",
    source="EXPRESS-COG-CARTO-TERRITOIRE",
    year=2022
)

# Filtrage France métropolitaine
departements = departements.loc[departements['INSEE_DEP'] < '970'] 
departements_proj = departements.to_crs(epsg=2154)  # Reprojection en Lambert-93


In [6]:
import pandas as pd
import os
import sys

# Définition du nom de fichier que nous avons créé précédemment
NOM_FICHIER_DESIRE = "C:\\Users\\ZEJLI\\Projet 5A\\bdd_temperature\\data_2000_2007.csv"
chemin_complet = os.path.join(os.getcwd(), NOM_FICHIER_DESIRE)

# --- 1. Importer le fichier CSV ---
try:
    # Lecture du fichier CSV
    # Nous utilisons 'index_col=0' car l'exportation précédente (index=True) 
    # a généralement transformé l'ancien MultiIndex (time, lat, lon) en première colonne.
    df_puy_de_dome = pd.read_csv(chemin_complet, index_col=0) 
    print(f"✅ Fichier '{NOM_FICHIER_DESIRE}' importé avec succès dans 'df_puy_de_dome'.")

except FileNotFoundError:
    print(f"❌ Erreur : Le fichier CSV est introuvable à l'emplacement : {chemin_complet}")
    print("Veuillez vous assurer que le script d'exportation a été exécuté correctement.")
    sys.exit(1)
except Exception as e:
    print(f"❌ Erreur lors de l'importation du CSV : {e}")
    sys.exit(1)


✅ Fichier 'C:\Users\ZEJLI\Projet 5A\bdd_temperature\data_2000_2007.csv' importé avec succès dans 'df_puy_de_dome'.


In [7]:
df_puy_de_dome.head(10)

,step,latitude,longitude,number,surface,valid_time,d2m,t2m
time,,,,,,,,
2000-01-01,0 days 01:00:00,45.7,3.1,0,0.0,2000-01-01 01:00:00,271.18793,272.33392
2000-01-01,0 days 01:00:00,45.7,3.2,0,0.0,2000-01-01 01:00:00,271.40472,272.56050
2000-01-01,0 days 01:00:00,45.6,3.1,0,0.0,2000-01-01 01:00:00,271.04535,272.10150
2000-01-01,0 days 01:00:00,45.6,3.2,0,0.0,2000-01-01 01:00:00,271.29535,272.36713
2000-01-01,0 days 02:00:00,45.7,3.1,0,0.0,2000-01-01 02:00:00,271.10522,272.25275
2000-01-01,0 days 02:00:00,45.7,3.2,0,0.0,2000-01-01 02:00:00,271.31226,272.45000
2000-01-01,0 days 02:00:00,45.6,3.1,0,0.0,2000-01-01 02:00:00,270.99000,272.06525
2000-01-01,0 days 02:00:00,45.6,3.2,0,0.0,2000-01-01 02:00:00,271.21070,272.27618
2000-01-01,0 days 03:00:00,45.7,3.1,0,0.0,2000-01-01 03:00:00,270.97955,272.12073


In [8]:
# Cr"ation du dataframe geopandas
gdf_geo_puy_de_dome = gpd.GeoDataFrame(
    df_puy_de_dome,
    geometry=gpd.points_from_xy(df_puy_de_dome['longitude'], df_puy_de_dome['latitude']),
    crs="EPSG:4326"
)
gdf_geo_puy_de_dome = gdf_geo_puy_de_dome.to_crs(departements_proj.crs)

In [9]:
gdf_geo_puy_de_dome.index

Index(['2000-01-01', '2000-01-01', '2000-01-01', '2000-01-01', '2000-01-01',
       '2000-01-01', '2000-01-01', '2000-01-01', '2000-01-01', '2000-01-01',
       ...
       '2007-12-01', '2007-12-01', '2007-12-01', '2007-12-01', '2007-12-01',
       '2007-12-01', '2007-12-01', '2007-12-01', '2007-12-01', '2007-12-01'],
      dtype='object', name='time', length=9216)

In [10]:
gdf_geo_puy_de_dome.columns

Index(['step', 'latitude', 'longitude', 'number', 'surface', 'valid_time',
       'd2m', 't2m', 'geometry'],
      dtype='object')

In [11]:
# On viens prendre les données d'une année en particulier, par exemple 2000 et 2001
gdf_geo_puy_de_dome.index = pd.to_datetime(gdf_geo_puy_de_dome.index)
gdf_2000 = gdf_geo_puy_de_dome[gdf_geo_puy_de_dome.index.year == 2000]
gdf_geo_puy_de_dome.index = pd.to_datetime(gdf_geo_puy_de_dome.index)
gdf_2007 = gdf_geo_puy_de_dome[gdf_geo_puy_de_dome.index.year == 2007]

gdf_2000.index = gdf_2000.index.strftime("%d/%m")
gdf_2007.index = gdf_2007.index.strftime("%d/%m")

In [12]:
# calcule de la différence entre les 2 années en conservant la latitude et longitude associées
df_diff = gdf_2000[["longitude", "latitude","geometry"]].copy()
df_diff["d2m_diff"] = gdf_2000["d2m"] - gdf_2007["d2m"]
gdf_geo_puy_de_dome = gpd.GeoDataFrame(
    df_diff,
    geometry=gpd.points_from_xy(df_diff['longitude'], df_diff['latitude']),
    crs="EPSG:4326"
)

In [13]:
df_diff.head()

,longitude,latitude,geometry,d2m_diff
time,,,,
01/01,3.1,45.7,POINT (707781.484 6511163.615),-3.09477
01/01,3.2,45.7,POINT (715562.955 6511178.397),-3.30182
01/01,3.1,45.6,POINT (707795.548 6500058.454),-3.02057
01/01,3.2,45.6,POINT (715591.083 6500073.263),-3.18658
01/01,3.1,45.7,POINT (707781.484 6511163.615),-2.99192


In [14]:
# calcule de la différence entre les 2 années en conservant la latitude et longitude associées
df_diff2 = gdf_2000[["longitude", "latitude","geometry"]].copy()
df_diff2["t2m_diff"] = gdf_2000["t2m"] - gdf_2007["t2m"]
gdf_geo_puy_de_dome = gpd.GeoDataFrame(
    df_diff2,
    geometry=gpd.points_from_xy(df_diff2['longitude'], df_diff2['latitude']),
    crs="EPSG:4326"
)

In [15]:
df_diff2.head()

,longitude,latitude,geometry,t2m_diff
time,,,,
01/01,3.1,45.7,POINT (707781.484 6511163.615),-3.44898
01/01,3.2,45.7,POINT (715562.955 6511178.397),-3.62084
01/01,3.1,45.6,POINT (707795.548 6500058.454),-3.34155
01/01,3.2,45.6,POINT (715591.083 6500073.263),-3.44311
01/01,3.1,45.7,POINT (707781.484 6511163.615),-3.36737


In [22]:
#FILTRAGE SPATIAL SUR LE PUY-DE-DÔME
departement_63 = departements_proj[departements_proj['INSEE_DEP'] == '63'].copy()

points_diff_63 = gpd.sjoin(
    df_diff, 
    departement_63[['INSEE_DEP', 'geometry']],
    predicate="within"
)

points_diff_63 = points_diff_63.to_crs(epsg=4326)
departement_63 = departement_63.to_crs(epsg=4326)

In [23]:
points_diff_63.head()

,longitude,latitude,geometry,d2m_diff,index_right,INSEE_DEP
time,,,,,,
01/01,3.1,45.7,POINT (3.1 45.7),-3.09477,18,63
01/01,3.2,45.7,POINT (3.2 45.7),-3.30182,18,63
01/01,3.1,45.6,POINT (3.1 45.6),-3.02057,18,63
01/01,3.2,45.6,POINT (3.2 45.6),-3.18658,18,63
01/01,3.1,45.7,POINT (3.1 45.7),-2.99192,18,63


In [24]:
# =================================================================
# 4. CALCUL DU CENTRE POUR LA CARTE
# =================================================================

bounds = departement_63.total_bounds  # [minx, miny, maxx, maxy]
CENTER_LON = (bounds[0] + bounds[2]) / 2
CENTER_LAT = (bounds[1] + bounds[3]) / 2


In [26]:
import branca.colormap as cm



# Création d'un colormap 
colormap = cm.linear.RdYlBu_11.scale(
    df_diff["d2m_diff"].min(),
    df_diff["d2m_diff"].max()
)
# Normaliser les valeurs
min_val = df_diff["d2m_diff"].min()
max_val = df_diff["d2m_diff"].max()


for dates in points_diff_63.index:
    m = folium.Map(
            location=[CENTER_LAT, CENTER_LON], 
            zoom_start=9, 
            tiles="CartoDB positron"
            )
    # Bordure du département
    folium.GeoJson(
        departement_63,
        style_function=lambda x: {
            'fillColor': 'none',
            'color': 'red',
            'weight': 2,
            'fillOpacity': 0
        },
        name="Puy-de-Dôme (63)"
    ).add_to(m)
    colormap.add_to(m)
    for i in range(len(points_diff_63.loc[dates, "d2m_diff"])):
        val = points_diff_63.loc[dates, "d2m_diff"][i]
        lat = points_diff_63.loc[dates, "latitude"][i]
        lon = points_diff_63.loc[dates, "longitude"][i]
        
        color = colormap(val- min_val / (max_val - min_val))
        folium.CircleMarker(
            location=[lat, lon],
            radius=4,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.8
        ).add_to(m)
        
    dates_formatted = dates.replace("/", ".")
    m.save(f"carte_d2m_2000_2007/carte_puy_de_dome_difference{dates_formatted}.html")




C:\Users\ZEJLI\AppData\Local\Temp\ipykernel_17392\927001375.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  val = points_diff_63.loc[dates, "d2m_diff"][i]
C:\Users\ZEJLI\AppData\Local\Temp\ipykernel_17392\927001375.py:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lat = points_diff_63.loc[dates, "latitude"][i]
C:\Users\ZEJLI\AppData\Local\Temp\ipykernel_17392\927001375.py:36: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lon

In [33]:
import pandas as pd
import geopandas as gpd
import folium
import branca.colormap as cm

# 1. Créer df_diff2 avec valid_time
df_diff2 = gdf_2000[["longitude", "latitude", "geometry", "valid_time"]].copy()
df_diff2["t2m_diff"] = gdf_2000["t2m"] - gdf_2007["t2m"]

# 2. CRS
df_diff2 = df_diff2.to_crs(epsg=4326)
departement_63 = departement_63.to_crs(epsg=4326)

# 3. Filtrage spatial sur Puy-de-Dôme
points_diff_t2m_63 = gpd.sjoin(
    df_diff2,
    departement_63[['INSEE_DEP','geometry']],
    predicate="within"
)
points_diff_t2m_63 = points_diff_t2m_63.reset_index(drop=True)

# 4. Filtrer janvier
points_diff_t2m_63['valid_time'] = pd.to_datetime(points_diff_t2m_63['valid_time'])
points_diff_janvier = points_diff_t2m_63[points_diff_t2m_63['valid_time'].dt.month == 1]

print(f"Nombre de points pour janvier : {len(points_diff_janvier)}")


# -------------------------------------------------------------------
# 3. Création de la carte
# -------------------------------------------------------------------
# Centre de la carte
minx, miny, maxx, maxy = departement_63.total_bounds
CENTER_LAT = (miny + maxy)/2
CENTER_LON = (minx + maxx)/2

m = folium.Map(location=[CENTER_LAT, CENTER_LON], zoom_start=9, tiles="CartoDB positron")

# Ajouter le contour du département
folium.GeoJson(departement_63).add_to(m)

# Colormap
colormap = cm.linear.RdYlBu_11.scale(points_diff_janvier['t2m_diff'].min(),
                                     points_diff_janvier['t2m_diff'].max())
colormap.caption = "Différence de température t2m (2000 - 2007)"
colormap.add_to(m)

# -------------------------------------------------------------------
# 4. Ajouter les points ERA5
# -------------------------------------------------------------------
for idx, row in points_diff_janvier.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=3,
        color=colormap(row['t2m_diff']),
        fill=True,
        fill_color=colormap(row['t2m_diff']),
        fill_opacity=0.8,
        stroke=False
    ).add_to(m)

folium.LayerControl().add_to(m)

# -------------------------------------------------------------------
# 5. Sauvegarde de la carte
# -------------------------------------------------------------------
m.save("heatmap_t2m_janvier_puy_de_dome.html")
print("✅ Carte sauvegardée : heatmap_t2m_janvier_puy_de_dome.html")


Nombre de points pour janvier : 96
✅ Carte sauvegardée : heatmap_t2m_janvier_puy_de_dome.html
